In [30]:
import numpy as np
import simpy
from collections import Counter
from tqdm import tqdm_notebook

processList = list()

class AMIDOLRateReward():
    def __init__(self):
        self.rewards = dict()
    
    def accumulateReward(self, params):
        return(0.0)
    
    def getDelay(self, params):
        return(np.Infinity)
    
    def getSample(self, params):
        return(np.Infinity)
    
    def isEnabled(self, params):
        return(True)

        
class AMIDOLEvent():
    def getName(self):
        return("GenericEvent")
    
    def getRate(self, params):
        return(1.0)
    
    def getDelay(self, params):
        delay = np.random.exponential(self.getRate(params))
        return(delay)
    
    def isEnabled(self, params):
        return(True)
    
    def fireEvent(self, params):
        return(params)
    
def dummyFire(self, params):
    return(params)
        
class AMIDOLSim():
    def __init__(self, until=10.0):
        self.clock = 0.0
        self.until = until
        self.eventList = []
        self.rateRewardList = []
    
    def registerEvent(self, event):
        self.eventList.append(event)
        
    def registerRateReward(self, reward):
        self.rateRewardList.append(reward)

    def stepSim(self, params):
        nextEventTime = np.Infinity
        nextEventFire = dummyFire
        nextRewardTime = np.Infinity
        nextRewardAccum = dummyFire
        
        for event in self.eventList:
            if (event.isEnabled(params)):
                delay = event.getDelay(params)
                if ((delay + self.clock) < nextEventTime):
                    nextEventTime = delay + self.clock
                    nextEventFire = event.fireEvent
        
        for reward in self.rateRewardList:
            sample = reward.getSample(params)
            if ((sample) < nextRewardTime):
                nextRewardTime = sample
                nextRewardAccum = reward.accumulateReward
                
        if (nextEventTime < nextRewardTime):
            self.clock = nextEventTime
            nextEventFire(params)
        else:
            self.clock = nextRewardTime
            nextRewardAccum(self.clock, params)
    
    def runSim(self, params):
        self.clock = 0.0
        while((not self.eventList == []) and (not self.rateRewardList == []) and (self.clock < self.until)):
            self.stepSim(params)

In [34]:
class AMIDOLParameters():
    def __init__(self):
        self.S_Pcinfect_S = 51999999
        self.ScinfectcI_Scinfect_IcI_Pccure_I = 1
        self.ScinfectcIccure_RcR_P = 0
        self.beta = 1.0/3.0*1.24
        self.gamma = 1.0/3.0
        
class infectEvent(AMIDOLEvent):
    def getName(self):
        return("InfectEvent")
    
    def getRate(self, v):
        rate = 1.0 / (v.beta*v.S_Pcinfect_S*v.ScinfectcI_Scinfect_IcI_Pccure_I/(v.S_Pcinfect_S+v.ScinfectcI_Scinfect_IcI_Pccure_I+v.ScinfectcIccure_RcR_P))
        return(rate)
    
    def isEnabled(self, v):
        return((v.beta*v.S_Pcinfect_S * v.ScinfectcI_Scinfect_IcI_Pccure_I) > 0.0)
    
    def fireEvent(self, v):
        v.S_Pcinfect_S -= 1.0
        v.ScinfectcI_Scinfect_IcI_Pccure_I += 1.0
        
class cureEvent(AMIDOLEvent):
    
    def getName(self):
        return("CureEvent")
    
    def getRate(self, v):
        rate = 1.0/(v.gamma*v.ScinfectcI_Scinfect_IcI_Pccure_I)
        return(rate)
    
    def isEnabled(self, v):
        return(v.gamma*v.ScinfectcI_Scinfect_IcI_Pccure_I > 0.0)
    
    def fireEvent(self, v):
        v.ScinfectcI_Scinfect_IcI_Pccure_I -= 1.0
        v.ScinfectcIccure_RcR_P += 1.0
        
class rvIRateReward(AMIDOLRateReward):
    def __init__(self):
        self.rewards = dict()
        self.samplePoints = [0.0, 5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0, 40.0, 45.0, 50.0, 55.0, 60.0, 65.0, 70.0, 75.0, 80.0, 85.0, 90.0, 95.0, 100.0]
        self.delays = list()
        self.delays.append(self.samplePoints[0])
        idx = 1
        lastX = self.samplePoints[0]
        for x in self.samplePoints[1:]:
            self.delays.append(x - lastX)
            lastX = x
    
    def accumulateReward(self, now, params):
        self.delays.pop(0)
        self.samplePoints.pop(0)
        self.rewards[now] = params.ScinfectcI_Scinfect_IcI_Pccure_I
        
    
    def getDelay(self, params):
        if (self.delays):
            return(self.delays[0])
        else:
            return(np.Infinity)
        
    def getSample(self, params):
        if (self.samplePoints):
            return(self.samplePoints[0])
        else:
            return(np.Infinity)
        
maxRuns = 1000

rvICounter = dict()

for trace in tqdm_notebook(range(0, maxRuns)):
    params = AMIDOLParameters()
    cure = cureEvent()
    infect = infectEvent()
    rvI = rvIRateReward()

    sim = AMIDOLSim(until=100.0)
    sim.registerEvent(cure)
    sim.registerEvent(infect)
    sim.registerRateReward(rvI)
    sim.runSim(params)
    
    if not(rvICounter):
        for key in rvI.rewards:
            rvICounter[key] = rvI.rewards[key] / float(maxRuns)
    else:
        for key in rvI.rewards:
            rvICounter[key] += rvI.rewards[key] / float(maxRuns)
    
print sorted(rvICounter.items())

[(0.0, 1.0000000000000007), (5.0, 1.4269999999999916), (10.0, 1.9579999999999924), (15.0, 2.832999999999993), (20.0, 4.353999999999996), (25.0, 6.475999999999995), (30.0, 9.69599999999999), (35.0, 14.584999999999997), (40.0, 21.83599999999998), (45.0, 32.55399999999999), (50.0, 49.14899999999999), (55.0, 72.93300000000004), (60.0, 108.06599999999996), (65.0, 160.64999999999995), (70.0, 239.1520000000001), (75.0, 358.06999999999994), (80.0, 535.3840000000002), (85.0, 795.4040000000007), (90.0, 1184.5929999999996), (95.0, 1758.211999999999), (100.0, 2605.2820000000006)]


In [ ]:
counter = Counter()
for trace in range(0, 100):
    env.run(until=rvI.samplePoints[-1])
    results = 